This is my first try at custom rendering for an object in Jupyter notebook: a character grid for use in Advent of Code.

For this, I went with HTML rendering.  SVG is the other option that looks attractive.  I wrote my own HTML builder, because I didn't need much, and didn't see any obvious libraries to use.

References:
 - [Integrating your objects with IPython](https://ipython.readthedocs.io/en/stable/config/shell_mimerenderer.html)

In [195]:
import html
import numpy as np

In [196]:
def flatten(x):
    """Yields all of the things in a (possibly nested) structure of lists and tuples"""
    if isinstance(x, (list, tuple)):
        for a in x:
            for b in flatten(a):
                yield b
    else:
        yield x  

def style(**kwargs):
    """Generate an (attr_name, attr_value) pair for style information.

    Input is keyword arguments, with the keys being style names, and the values being the setting.

    Use "_" instead of "-" in style names so they work with python, like this:
       style(background_color="blue")
    """
    def fix_name(k):
        return k.replace("_", "-")
    return ("style", "; ".join(f"{fix_name(k)}: {html.escape(v)}" for (k, v) in kwargs.items()))

class Elem:
    """An HTML element to render

    Contents:
       name - the name of the element, such as "p" or "div"
       attrs - a dictionary holding attribute names and values (passed in as list of key/value pairs)
       children - a sequence of child nodes; can be nested tuples/lists
       text - text that goes in this element; will go after children
    """
    def __init__(self, name, attrs, children, text=None):
        self.name = name
        self.attrs = dict(attrs)
        self.children = children
        self.text = text

    def render(self):
        return "".join(self.render_lines(0))
        
    def render_lines(self, indent):
        """Yields all of the lines in the HTML representation of this object"""
        def line(n, s):
            return "  " * n + s + "\n"
        yield line(indent, f"<{self.name}{self.render_attrs()}>")
        for c in flatten(self.children):
            for s in c.render_lines(indent + 1):
                yield s
        if self.text is not None:
            yield line(indent, html.escape(self.text))
        yield line(indent, f"</{self.name}>")

    def render_attrs(self):
        def render_attr(attr_name):
            return " " + attr_name + "=" + "\"" + self.attrs[attr_name] + "\""
        return "".join(render_attr(attr_name) for attr_name in sorted(self.attrs.keys()))

In [197]:
def to_pair(x):
    """Convert a container holding two things into a pair, if it's not already

    Works on numpy arrays and lists.
    """
    if isinstance(x, tuple) and len(x) == 2:
        return x
    else:
        a, b = x
        return (a, b)

def neighbors(p):
    """Yield all of the neighbors of a location in a grid"""
    (x, y) = p
    for dx in (-1, 0, 1):
        for dy in (-1, 0, 1):
            if dx != 0 or dy != 0:
                yield (x + dx, y + dy)

def left(p):
    x, y = p
    return (x - 1, y)

def right(p):
    x, y = p
    return (x + 1, y)

In [198]:
class Grid:
    """A two-dimensional grid.

    Stored as a dict (map), with keys that are pairs of ints, and values that are one-character strings.

    The keys are not numpy arrays, because those are not hashable.
    """
    def __init__(self, data=None, color_fcn=None):
        self.data = {} if data is None else data
        self.color_fcn = (lambda k, v: None) if color_fcn is None else color_fcn

    def bounds(self):
        """Return the bounds along x and y axes: ((xmin, xmax), (ymin, ymax))

        Lower bounds are inclusive, upper bounds are exclusive, so the work with range().
        """
        xmin = min(x for (x, _) in self.data.keys())
        xmax = max(x+1 for (x, _) in self.data.keys())
        ymin = min(y for (_, y) in self.data.keys())
        ymax = max(y+1 for (_, y) in self.data.keys())
        return ((xmin, xmax), (ymin, ymax))

    def items(self):
        """Generates all of the (k, v) pairs that have data in the grid"""
        return self.data.items()

    def get(self, k, dflt):
        """Returns the value at the given location, or the default if there is nothing"""
        return self.data.get(k, dflt)

    @classmethod
    def from_string(cls, s, count_from=0, **kwargs):
        """Parses a grid from a multi-line string.

        `count_from` defaults to 0, meaning 0-based indexing.  Some puzzles want 1-based; passing in 1 will do that.
        """
        data = dict(
            ((x + count_from, y + count_from), v)
            for (y, line) in enumerate(s.splitlines())
            for (x, v) in enumerate(line)
        )
        return cls(data, **kwargs)

    @classmethod
    def from_file(cls, file_name, **kwargs):
        """Parsed a grid contained in a file"""
        with open(file_name, "r") as f:
            return cls.from_string(f.read(), **kwargs)

    def _repr_html_(self):
        """Format in HTML for display in a notebook."""
        (x0, x1), (y0, y1) = self.bounds()
        html = Elem("table", [], [
            self.header_row(x0, x1),
            [self.row_html(y, x0, x1) for y in range(y0, y1)]
        ])
        return html.render()

    def header_row(self, x0, x1):
        return Elem("tr", [style(background_color="#ccc")], [
            Elem("th", [], []),
            [Elem("th", [style(transform="rotate(90deg)")], [], str(x)) for x in range(x0, x1)]
        ])
        
    def row_html(self, y, x0, x1):
        return Elem("tr", [], [ 
            Elem("th", [style(background_color="#ccc")], [], str(y)),
            [self.cell_html(x, y) for x in range (x0, x1)]
        ])

    def cell_html(self, x, y):
        v = self.data.get((x, y), " ")
        color = self.color_fcn((x, y), v) or "white"
        return Elem("td", [style(font_family="monospace", background_color=color)], [], v)

In [199]:
def color_cell(k, v):
    if v.isdigit():
        return "pink"
    elif v != ".":
        return "orange"
    else:
        return "white"
        
test_problem = Grid.from_file("test.txt", color_fcn=color_cell)
test_problem

,0,1,2,3,4,5,6,7,8,9
0,4,6,7,.,.,1,1,4,.,.
1,.,.,.,*,.,.,.,.,.,.
2,.,.,3,5,.,.,6,3,3,.
3,.,.,.,.,.,.,#,.,.,.
4,6,1,7,*,.,.,.,.,.,.
5,.,.,.,.,.,+,.,5,8,.
6,.,.,5,9,2,.,.,.,.,.
7,.,.,.,.,.,.,7,5,5,.
8,.,.,.,$,.,*,.,.,.,.
9,.,6,6,4,.,5,9,8,.,.


In [200]:
# To find all of the part numbers, let's find all of the digits that start part numbers, and work from there.

def is_symbol(c):
    return c not in ". " and not c.isdigit()
    
def all_part_numbers(grid):

    def has_digit(p):
        return grid.get(p, ' ').isdigit()

    def has_symbol(p):
        return is_symbol(grid.get(p, ' '))
    
    def starts_number(p):
        """Is the given point one that is the beginning of a number"""
        return has_digit(p) and not has_digit(left(p))
        
    def number_points(p):
        """Returns a list of the points that hold a number.  `p` is the start of the number"""
        result = []
        while has_digit(p):
            result.append(p)
            p = right(p)
        return result

    def is_part_number(ps):
        """Is the number at the given locations a part number?  It is if it's next to a symbol."""
        return any(
            has_symbol(n)
            for p in ps
            for n in neighbors(p)
        )

    for (p, _) in grid.items():
        if starts_number(p):
            ps = number_points(p)
            if is_part_number(ps):
                yield ps

def part_number_value(ps, grid):
    n = 0
    for p in ps:
        n = n * 10 + int(grid.get(p, ' '))
    return n

def part1(file_name):
    problem = Grid.from_file(file_name)
    return sum(part_number_value(ps, problem) for ps in all_part_numbers(problem))

In [201]:
part1("test.txt")
    

4361

In [202]:
part1("input.txt")

539637

In [203]:
def is_next_to_part_number(p, pn):
    """Is the given point next to one of the digits of the part number?"""
    return any(
        p == n
        for p2 in pn
        for n in neighbors(p2)
    )

def find_gears(problem):
    """Generate a list of triples: (gear_location, part_a, part_b)
    
    These are the locations of gears, and the two part numbers next to them.
    """
    part_numbers = list(all_part_numbers(problem))
    for p, c in problem.items():
        if is_symbol(c):
            adjacent_part_numbers = [
                pn
                for pn in part_numbers 
                if is_next_to_part_number(p, pn)
            ]
            if len(adjacent_part_numbers) == 2:
                pn1, pn2 = adjacent_part_numbers 
                yield (p, pn1, pn2)

def color_gears(problem):
    gear_locs = set(
        p
        for (p, _, _) in find_gears(problem)
    )
    def f(k, v):
        if k in gear_locs:
            return "red"
        else:
            return color_cell(k, v)
    return f

test_problem.color_fcn = color_gears(test_problem)
test_problem
            

,0,1,2,3,4,5,6,7,8,9
0,4,6,7,.,.,1,1,4,.,.
1,.,.,.,*,.,.,.,.,.,.
2,.,.,3,5,.,.,6,3,3,.
3,.,.,.,.,.,.,#,.,.,.
4,6,1,7,*,.,.,.,.,.,.
5,.,.,.,.,.,+,.,5,8,.
6,.,.,5,9,2,.,.,.,.,.
7,.,.,.,.,.,.,7,5,5,.
8,.,.,.,$,.,*,.,.,.,.
9,.,6,6,4,.,5,9,8,.,.


In [204]:
def part2(file_name):
    problem = Grid.from_file(file_name)
    return sum(
        part_number_value(pn1, problem) * part_number_value(pn2, problem)
        for (_, pn1, pn2) in find_gears(problem)
    )

In [205]:
part2("test.txt")

467835

In [206]:
part2("input.txt")

82818007